In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim

pd.set_option("display.max_columns", None)

In [ ]:
from sklearn.datasets import load_breast_cancer


torch.manual_seed(1331)

data = load_breast_cancer()

X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.float32).unsqueeze(-1)

n = int(len(X)*0.8)

X_train = X[:n]
X_test = X[n:]
y_train = y[:n]
y_test = y[n:]

print(f"Train: {X_train.shape} and {y_train.shape}")
print(f"Test: {X_test.shape} and {y_test.shape}")



class MiniNeuron(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(X_train.shape[1], X_train.shape[1]),
            nn.Linear(X_train.shape[1], 1),
            nn.Sigmoid()
        )
    
    def forward(self, X):
        return self.net(X)
    
# training the MLP
model = MiniNeuron()

critation = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
eval_interval = epochs * 0.1

for i in range(epochs):
    y_pred = model(X_train)
    loss = critation(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % eval_interval == 0:
        print(f'iter: {i+1} loss={loss.item()}')

Train: torch.Size([455, 30]) and torch.Size([455, 1])
Test: torch.Size([114, 30]) and torch.Size([114, 1])
iter: 1 loss=0.6931472420692444
iter: 101 loss=0.6931472420692444
iter: 201 loss=0.6931472420692444
iter: 301 loss=0.6931472420692444
iter: 401 loss=0.6931472420692444
iter: 501 loss=0.6931472420692444
iter: 601 loss=0.6931472420692444
iter: 701 loss=0.6931472420692444
iter: 801 loss=0.6931472420692444
iter: 901 loss=0.6931472420692444


In [154]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing(return_X_y=True)

scaler = StandardScaler()

X = torch.tensor(scaler.fit_transform(data[0]), dtype=torch.float)
y = torch.tensor(data[1], dtype=torch.float)

# defining the shape
n, n_features = X.shape

# spliting the dataset
sps = int(n*.8)
X_train = X[:sps]
X_test = X[sps:]
y_train = y[:sps]
y_test = y[sps:]

# model config
learning_rate = 1e-2
epochs = 5000

# building our MLP
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.ly1 = nn.Linear(n_features, 64)
        self.ly2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    
    def forward(self, X):
        x= self.relu(self.ly1(X))
        x = self.ly2(x)
        return x 
    


# initialing our NN
model = MLP()

# loss and optimizers
critation = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for k in range(epochs):
    y_pred = model(X_train).squeeze()
    loss = critation(y_pred, y_train)

    # applying optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (k+1) % 1000 == 0:
        print(f'Iter: {k+1}, loss={loss.item()}')


model.eval()

with torch.no_grad():
    y_pred = model(X_test).squeeze()

mse = torch.mean((y_pred - y_test)**2).item()
rmse = mse**0.5

print(f"Test MSE: {mse:.4f}")

Iter: 1000, loss=0.25615429878234863
Iter: 2000, loss=0.23632727563381195
Iter: 3000, loss=0.2280023992061615
Iter: 4000, loss=0.22484996914863586
Iter: 5000, loss=0.2225530743598938
Test MSE: 0.6873


In [156]:
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Load data
X_np, y_np = fetch_california_housing(return_X_y=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)

X = torch.tensor(X_scaled, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).reshape(-1, 1)

# Add bias column (intercept)
ones = torch.ones((X.shape[0], 1))
X_bias = torch.cat([ones, X], dim=1)

# Normal Equation: theta = (X^T X)^(-1) X^T y
theta = torch.inverse(X_bias.T @ X_bias) @ X_bias.T @ y

print("Weight vector (including bias):")
print(theta)

# Split data (same 80/20 split as before)
sps = int(0.8 * X.shape[0])
X_test = X_bias[sps:]
y_test = y[sps:]

# Predict
y_pred = X_test @ theta

# Compute MSE
mse = torch.mean((y_pred - y_test)**2).item()
print(f"Closed-form Test MSE: {mse:.4f}")

Weight vector (including bias):
tensor([[ 2.0686],
        [ 0.8296],
        [ 0.1188],
        [-0.2655],
        [ 0.3057],
        [-0.0045],
        [-0.0393],
        [-0.8998],
        [-0.8705]])
Closed-form Test MSE: 0.4753


In [147]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Test MSE: {mean_squared_error(y_test, y_pred):.4f}")

Test MSE: 0.4947


In [ ]:
net = nn.Sequential(
    nn.Linear(12, 6),
    nn.Linear(6, 1),
)


# if there is no bias, no need for adding 
print(len(list(net.parameters())))
print(f"my calculation: {(12*6+6) + (6*1+1)}")

res = [p.numel() for p in net.parameters() if p.requires_grad]

print(res)
print(sum(res))
        

4
my calculation: 85
[72, 6, 6, 1]
85


In [164]:
assert torch.randn(2, 3, 3).numel() == 2*3*3, "failed"

In [165]:
torch.randn(3, 4, 3)

tensor([[[-2.0391, -0.2416, -1.7920],
         [ 0.4204, -0.6147,  0.2263],
         [ 1.5441, -1.3303, -1.3287],
         [-0.4162,  0.4767,  0.6769]],

        [[ 0.2300, -0.2331,  2.1463],
         [ 0.9718, -0.3368, -0.7853],
         [-0.7826, -0.4238,  0.0660],
         [ 2.2129, -0.8283,  1.6527]],

        [[ 0.7336, -0.6972,  0.3850],
         [-0.0322,  1.0844,  0.8005],
         [ 0.1479, -1.4065,  0.3000],
         [-1.6717,  1.2735,  0.4505]]])